In [1]:
import tensorflow as tf

In [2]:
from tensorflow import keras

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator #
from tensorflow.keras.models import Sequential #
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D 
#Dense=computational unit make less
#Dropout = Numerical unit drop

In [4]:
#cd /path
#!unzip /path or archive.zip
num_classes = 2 #cat and dog
img_row = 64
img_col = 64 #image size change to matrics from square image
train_dataset = "/content/drive/MyDrive/archive.zip (Unzipped Files)/training_set/training_set"
test_dataset = "/content/drive/MyDrive/archive.zip (Unzipped Files)/test_set/test_set"

In [5]:
#data augmentation
train_datagen = ImageDataGenerator(
rescale = 1./255, #normaliation
rotation_range = 30,
width_shift_range = 0.4,
height_shift_range = 0.4,
horizontal_flip = True,
fill_mode='nearest'
)

In [6]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_dataset,
    target_size=(img_row,img_col),
    batch_size = 32,
    class_mode = 'categorical', #multicalfication
    shuffle = True
)

Found 8005 images belonging to 2 classes.


In [9]:
validation_generator = train_datagen.flow_from_directory(
    test_dataset,
    target_size=(img_row,img_col),
    batch_size = 32, #if more batch fast
    class_mode = 'categorical', #categorical for multiimageclassification
    shuffle = True
)

Found 2025 images belonging to 2 classes.


In [10]:
model = Sequential() #feature map
#input layer for convolution
model.add(Conv2D(64,(3,3),padding='same',input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#hidden layer conv
model.add(Conv2D(32,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#model.add(Conv2D(16,(3,3),padding='same'))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size = (2,2)))
#model.add(Dropout(0.25))

#flatten conv
model.add(Flatten())#single layer 1D
model.add(Dense(256)) #only few send
model.add(Activation('relu'))
model.add(Dropout(0.5)) #size decrease
model.summary()

#output shape
# 64 by 64 image of 64 feature map

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 64)        1792      
                                                                 
 activation (Activation)     (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        18464     
                                                                 
 activation_1 (Activation)   (None, 32, 32, 32)        0         
                                                        

In [11]:
#neural network
model.add(Dense(units=256, activation = 'relu'))
model.add(Dense(units=128, activation = 'relu'))
model.add(Dense(units=32, activation = 'relu'))
model.add(Dense(units=16, activation = 'relu')) # only +
model.add(Dense(2))
model.add(Activation('sigmoid')) #0 or 1 output layer ma yes or no softmax do for probality
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 64)        1792      
                                                                 
 activation (Activation)     (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        18464     
                                                                 
 activation_1 (Activation)   (None, 32, 32, 32)        0         
                                                        

In [12]:
#comile before traing
#training
model.compile(loss='categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch = 800//32,
    epochs = 1, #do maximum epoch for better test
    validation_data= validation_generator,
    validation_steps = 202//32
)
#layer decrease

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if sys.path[0] == '':


25/25 [==============================] - 212s 8s/step - loss: 0.7186 - accuracy: 0.4975 - val_loss: 0.6929 - val_accuracy: 0.5156


In [13]:
model.save('/content/drive/MyDrive/archive.zip (Unzipped Files)/cat_vs_dog.h5')

In [15]:
#test
from tensorflow.python.keras.utils.generic_utils import class_and_config_for_serialized_keras_object
import numpy as np #to convert array
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

classifier = load_model('/content/drive/MyDrive/archive.zip (Unzipped Files)/cat_vs_dog.h5')
test_image = image.load_img('/content/drive/MyDrive/archive.zip (Unzipped Files)/test_set/test_set/dogs/dog.4001.jpg',target_size = (64,64,3))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
result = classifier.predict(test_image)
print(result)
#if result > 0.6:
 # print("This is dog")
#else:
  #print("This is cat")

[[0.6270496  0.74256116]]


In [18]:
#for multiimage
probability_of_dog = result[0][0]
probability_of_cat = result[0][1]
print("probability of cat ", probability_of_cat)
print("probability of dog ", probability_of_dog)

probability of cat  0.74256116
probability of dog  0.6270496
